In [30]:
import scipy.stats as st
import numpy as np

# Preparing the data for hypothesis tests

# Setting up the hypothesis tests

- Null hyp: No change in the ratio of positive because of the influencer's positive review
- Alt hyp: There is an increase in the ratio of positive reviews after influencer's positive review 

([citation](https://online.stat.psu.edu/statprogram/reviews/statistical-concepts/proportions#toggleSidebar))

In [26]:
# dummy values for now
rp0 = 0.5
rp1 = 0.9
n = 30
alpha = 0.05

In [27]:
# test statistic
z = (rp1-rp0)/ np.sqrt((rp0*(1-rp0))/n)
z

4.3817804600413295

In [28]:
pValue = 1 - st.norm.cdf(z)
pValue

5.8856695488440636e-06

In [29]:
if pValue < alpha :
    print('null rejected')
else : 
    print('cannot reject null')

null rejected
